<a href="https://colab.research.google.com/github/DanteAgarwal/meta_heuristics_optimzation_algorithm/blob/main/major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# creating dataset from kaggle

In [ ]:
import os

In [ ]:
os.environ['KAGGLE_USERNAME'] = "akashrathore714" # username from the json file
os.environ['KAGGLE_KEY'] = "aa9e778f52477a51041c324bc2e5dd3b" # key from the json file

In [ ]:
%mkdir ctscan_preprocessed
%cd /content/ctscan_preprocessed
%mkdir Non_covid_preprocessed
%mkdir covid_preprocessed

/content/ctscan_preprocessed


In [ ]:
%cd ..
! kaggle datasets download -d plameneduardo/sarscov2-ctscan-dataset

 99% 229M/230M [00:05<00:00, 36.4MB/s]
100% 230M/230M [00:05<00:00, 45.9MB/s]


In [ ]:
!mkdir ctscans
!unzip sarscov2-ctscan-dataset.zip -d ctscans

Archive:  sarscov2-ctscan-dataset.zip
  inflating: ctscans/COVID/Covid (1).png  
  inflating: ctscans/COVID/Covid (10).png  
  inflating: ctscans/COVID/Covid (100).png  
  inflating: ctscans/COVID/Covid (1000).png  
  inflating: ctscans/COVID/Covid (1001).png  
  inflating: ctscans/COVID/Covid (1002).png  
  inflating: ctscans/COVID/Covid (1003).png  
  inflating: ctscans/COVID/Covid (1004).png  
  inflating: ctscans/COVID/Covid (1005).png  
  inflating: ctscans/COVID/Covid (1006).png  
  inflating: ctscans/COVID/Covid (1007).png  
  inflating: ctscans/COVID/Covid (1008).png  
  inflating: ctscans/COVID/Covid (1009).png  
  inflating: ctscans/COVID/Covid (101).png  
  inflating: ctscans/COVID/Covid (1010).png  
  inflating: ctscans/COVID/Covid (1011).png  
  inflating: ctscans/COVID/Covid (1012).png  
  inflating: ctscans/COVID/Covid (1013).png  
  inflating: ctscans/COVID/Covid (1014).png  
  inflating: ctscans/COVID/Covid (1015).png  
  inflating: ctscans/COVID/Covid (1016).png  
  i

# importing important modeles of python


In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt


import sys
import random
import warnings
warnings.filterwarnings('ignore')

import cv2
import os
from PIL import Image
from skimage.filters.rank import entropy
from skimage.morphology import disk

from scipy.signal import convolve2d
from skimage.segmentation import slic,mark_boundaries

## preprocessing of dataset images
### 1. clahe Algo (for contrast) .
### 2. cropping of image .
### 3. main driver code .
### (also we maked full data set in 512 , 512 resloution and grayscale) .

In [ ]:
def clahe_enhancer(img,demo=False):
    img = np.uint8(img*255)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(img)
    if demo:
        img_flattened = img.flatten()
        clahe_img_flattened = clahe_img.flatten()
        fig = plt.figure()
        rcParams['figure.figsize'] = 10,10

        plt.subplot(2, 2, 1)
        plt.imshow(img, cmap='bone')
        plt.title("Original CT-Scan")

        plt.subplot(2, 2, 2)
        plt.hist(img_flattened)
        plt.title("Histogram of Original CT-Scan")

        plt.subplot(2, 2, 3)
        plt.imshow(clahe_img, cmap='bone')
        plt.title("CLAHE Enhanced CT-Scan")

        plt.subplot(2, 2, 4)
        plt.hist(clahe_img_flattened)
        plt.title("Histogram of CLAHE Enhanced CT-Scan")

    return clahe_img

In [ ]:
import imutils
def cropper1(img):

  grassuian=cv2.GaussianBlur(img,(5,5),0)
  threshold_img=cv2.threshold(grassuian,45,255,cv2.THRESH_BINARY)[1]
  threshold_img = cv2.erode(threshold_img,None,iterations=2)
  threshold_img = cv2.dilate(threshold_img,None,iterations=2)

  contour = cv2.findContours(threshold_img.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  contour = imutils.grab_contours(contour)
  c=max(contour, key=cv2.contourArea)

  extreme_pnts_left=tuple(c[c[:,:,0].argmin()][0])
  extreme_pnts_right=tuple(c[c[:,:,0].argmax()][0])
  extreme_pnts_top=tuple(c[c[:,:,1].argmin()][0])
  extreme_pnts_bottom=tuple(c[c[:,:,1].argmax()][0])

  new_img =img[extreme_pnts_top[1]:extreme_pnts_bottom[1],extreme_pnts_left[0]:extreme_pnts_right[0]]
  return new_img

In [ ]:
from google.colab.patches import cv2_imshow
import glob
imdir = '/content/ctscans/COVID/'
ext = ['png']
files = []
cout=0
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
for file in files:
  img = cv2.imread(file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img1 = cv2.resize(img,(512,512),interpolation=cv2.INTER_AREA)
  Clahe_img1 = clahe_enhancer(img1)
  img1=cropper1(Clahe_img1)
  cout +=1
  cv2.imwrite("/content/ctscan_preprocessed/covid_preprocessed/covid_preposed_{0}.png".format(cout), img1)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
imdir = '/content/ctscans/non-COVID/'
ext = ['png']
files = []
cout=0
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
for file in files:
  img = cv2.imread(file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img1 = cv2.resize(img,(512,512),interpolation=cv2.INTER_AREA)
  Clahe_img1 = clahe_enhancer(img1)
  img1=cropper1(Clahe_img1)
  cout +=1
  cv2.imwrite('/content/ctscan_preprocessed/Non_covid_preprocessed/Non_covid_preposed_{0}.png'.format(cout), img1)

cv2.waitKey()
cv2.destroyAllWindows()

# feature extraction
### 1 GLCM
### 2 gabor
### 3 dwt
### 4 GLRLM
### 5 first order statistical
### 6 hog feature
### 7 histogram feature
### 8 multi histogram feature

In [ ]:
#glcm filter
"""
skimage.feature.greycomatrix(image, distances, angles, levels=None, symmetric=False, normed=False)
distances - List of pixel pair distance offsets.
angles - List of pixel pair angles in radians.
skimage.feature.greycoprops(P, prop)
prop: The property of the GLCM to compute.
{‘contrast’, ‘dissimilarity’, ‘homogeneity’, ‘energy’, ‘correlation’, ‘ASM’}
"""


#Capture images and labels into arrays.
#Start by creating empty lists.
train_images = []
train_labels = []
#for directory_path in glob.glob("cell_images/train/*"):
for directory_path in glob.glob("D:/major project/covid_data_spliting/train/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path)#Reading color images
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

#Do exactly the same for test/validation images
# test
test_images = []
test_labels = []
#for directory_path in glob.glob("cell_images/test/*"):
for directory_path in glob.glob("D:/major project/covid_data_spliting/val/*"):
    fruit_label = directory_path.split("//")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img1 = cv2.imread(img_path)
        test_images.append(img1)
        test_labels.append(fruit_label)


test_labels = np.array(test_labels)

#Encode labels from text (folder names) to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)
#If you only have one dataset then split here
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

# Normalize pixel values to between 0 and 1
#x_train, x_test = x_train / 255.0, x_test / 255.0

###################################################################
# FEATURE EXTRACTOR function
# input shape is (n, x, y, c) - number of images, x, y, and channels
def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):  #iterate through each file
        #print(image)

        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.

        img = dataset[image,:,:]
    ################################################################
    #START ADDING DATA TO THE DATAFRAME


         #Full image
        #GLCM = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        GLCM = greycomatrix(img, [1], [0])
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df['Energy'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df['Corr'] = GLCM_corr
        GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
        df['Diss_sim'] = GLCM_diss
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df['Homogen'] = GLCM_hom
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df['Contrast'] = GLCM_contr



        GLCM2 = greycomatrix(img, [3], [0])
        GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
        df['Energy2'] = GLCM_Energy2
        GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
        df['Corr2'] = GLCM_corr2
        GLCM_diss2 = greycoprops(GLCM2, 'dissimilarity')[0]
        df['Diss_sim2'] = GLCM_diss2
        GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
        df['Homogen2'] = GLCM_hom2
        GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
        df['Contrast2'] = GLCM_contr2


        GLCM3 = greycomatrix(img, [5], [0])
        GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
        df['Energy3'] = GLCM_Energy3
        GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
        df['Corr3'] = GLCM_corr3
        GLCM_diss3 = greycoprops(GLCM3, 'dissimilarity')[0]
        df['Diss_sim3'] = GLCM_diss3
        GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
        df['Homogen3'] = GLCM_hom3
        GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
        df['Contrast3'] = GLCM_contr3


        GLCM4 = greycomatrix(img, [0], [np.pi/4])
        GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
        df['Energy4'] = GLCM_Energy4
        GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
        df['Corr4'] = GLCM_corr4
        GLCM_diss4 = greycoprops(GLCM4, 'dissimilarity')[0]
        df['Diss_sim4'] = GLCM_diss4
        GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
        df['Homogen4'] = GLCM_hom4
        GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
        df['Contrast4'] = GLCM_contr4


        GLCM5 = greycomatrix(img, [0], [np.pi/2])
        GLCM_Energy5 = greycoprops(GLCM5, 'energy')[0]
        df['Energy5'] = GLCM_Energy5
        GLCM_corr5 = greycoprops(GLCM5, 'correlation')[0]
        df['Corr5'] = GLCM_corr5
        GLCM_diss5 = greycoprops(GLCM5, 'dissimilarity')[0]
        df['Diss_sim5'] = GLCM_diss5
        GLCM_hom5 = greycoprops(GLCM5, 'homogeneity')[0]
        df['Homogen5'] = GLCM_hom5
        GLCM_contr5 = greycoprops(GLCM5, 'contrast')[0]
        df['Contrast5'] = GLCM_contr5


        #Add more filters as needed
        #entropy = shannon_entropy(img)
        #df['Entropy'] = entropy


        #Append features from current image to the dataset
        image_dataset = image_dataset.append(df)

    return image_dataset
####################################################################
#Extract features from training images
image_features = feature_extractor(img)
image_features.to_csv('D:/major project/feature-extraction/features.csv')




In [ ]:
#GLRLM
import numpy as np

def glrlm_0(f, mask, grayLevel=5, runLength=5, skipFirstRow=True):
    degree0Matrix = np.zeros([grayLevel, runLength])
    counter = 0
    for y in range(f.shape[0]):
        for x in range(f.shape[1]):
            nowVal = f[y][x]
            if x + 1 >= f.shape[1]:
                nextVal = None
            else:
                nextVal = f[y][x + 1]
            if nextVal != nowVal and counter == 0:
                degree0Matrix[int(nowVal)][counter] += 1
            elif nextVal == nowVal:
                counter += 1
            elif nextVal != nowVal and counter != 0:
                degree0Matrix[int(nowVal)][counter] += 1
                counter = 0
    return degree0Matrix[1:,:] if skipFirstRow else degree0Matrix

def glrlm_90(f, mask, grayLevel=5, runLength=5, skipFirstRow=True):
    degree90Matrix = np.zeros([grayLevel, runLength])
    counter = 0
    for x in range(f.shape[1]):
        for y in range(f.shape[0]):
            nowVal = f[y][x]
            if y + 1 >= f.shape[0]:
                nextVal = None
            else:
                nextVal = f[y + 1][x]
            if nextVal != nowVal and counter == 0:
                degree90Matrix[int(nowVal)][counter] += 1
            elif nextVal == nowVal:
                counter += 1
            elif nextVal != nowVal and counter != 0:
                degree90Matrix[int(nowVal)][counter] += 1
                counter = 0
    return degree90Matrix[1:,:] if skipFirstRow else degree90Matrix

def glrlm_45(f, mask, grayLevel=5, runLength=5, skipFirstRow=True):
    degree45Matrix = np.zeros([grayLevel, runLength])
    for y in range(f.shape[0]):
        counter = 0
        i_range = max(f.shape)
        for i in range(i_range):
            y1 = y - i
            if i >= f.shape[1] or y1 < 0:
                break
            else:
                nowVal = f[y1][i]
            if y1 - 1 < 0 or i + 1 >= f.shape[1]:
                nextVal = None
            else:
                nextVal = f[y1 - 1][i + 1]
            if nextVal != nowVal and counter == 0:
                degree45Matrix[int(nowVal)][counter] += 1
            elif nextVal == nowVal:
                counter += 1
            elif nextVal != nowVal and counter != 0:
                degree45Matrix[int(nowVal)][counter] += 1
                counter = 0
    for x in range(f.shape[1]):
        if x == f.shape[1] - 1:
            break
        counter = 0
        i_range = max(f.shape)
        for i in range(i_range):
            y_i = -1 - i
            x_i = -1 + i - x
            if x_i >= 0 or y_i <= -1 - f.shape[0]:
                break
            else:
                nowVal = f[y_i][x_i]
            if y_i - 1 <= -(f.shape[0] + 1) or x_i + 1 >= 0:
                nextVal = None
            else:
                nextVal = f[y_i - 1][x_i + 1]
            if nextVal != nowVal and counter == 0:
                degree45Matrix[int(nowVal)][counter] += 1
            elif nextVal == nowVal:
                counter += 1
            elif nextVal != nowVal and counter != 0:
                degree45Matrix[int(nowVal)][counter] += 1
                counter = 0
    degree45Matrix[0,1:] = 0
    return degree45Matrix[1:,:] if skipFirstRow else degree45Matrix

def glrlm_135(f, mask, grayLevel=5, runLength=5, skipFirstRow=True):
    degree135Matrix = np.zeros([grayLevel, runLength])
    for y in range(f.shape[0]):
        counter = 0
        i_range = max(f.shape)
        for i in range(i_range):
            y1 = y + i
            if y1 >= f.shape[0] or i >= f.shape[1]:
                break
            else:
                nowVal = f[y1][i]
                if y1 >= f.shape[0] - 1 or i >= f.shape[1] - 1:
                    nextVal = None
                else:
                    nextVal = f[y1 + 1][i + 1]
                if nextVal != nowVal and counter == 0:
                    degree135Matrix[int(nowVal)][counter] += 1
                elif nextVal == nowVal:
                    counter += 1
                elif nextVal != nowVal and counter != 0:
                    degree135Matrix[int(nowVal)][counter] += 1
                    counter = 0
    for x in range(f.shape[1]):
        if x == 0:
            continue
        i_range = max(f.shape)
        counter = 0
        for i in range(i_range):
            x1 = x + i
            if i >= f.shape[0] or x1 >= f.shape[1]:
                break
            else:
                nowVal = f[i][x1]
            if i >= f.shape[0] - 1 or x1 >= f.shape[1] - 1:
                nextVal = None
            else:
                nextVal = f[i + 1][x1 + 1]
            if nextVal != nowVal and counter == 0:
                degree135Matrix[int(nowVal)][counter] += 1
            elif nextVal == nowVal:
                counter += 1
            elif nextVal != nowVal and counter != 0:
                degree135Matrix[int(nowVal)][counter] += 1
                counter = 0
    degree135Matrix[0,1:] = 0
    return degree135Matrix[1:,:] if skipFirstRow else degree135Matrix

def _apply_over_degree(function, x1, x2):
    if function == np.divide:
        x2 = x2 + 1e-16
    rows, cols, nums = x1.shape
    result = np.ndarray((rows, cols, nums))
    for i in range(nums):
        result[:, :, i] = function(x1[:, :, i], x2)
        result[result == np.inf] = 0
        result[np.isnan(result)] = 0
    return result

def _calculate_ij (rlmatrix):
    gray_level, run_length, _ = rlmatrix.shape
    I, J = np.ogrid[0:gray_level, 0:run_length]
    return I, J+1

def _calculate_s(rlmatrix):
    return np.apply_over_axes(np.sum, rlmatrix, axes=(0, 1))[0, 0]

def glrlm(f, mask, Ng=256):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else.
    Ng : int, optional
        Image number of gray values. The default is 256.
    Returns
    -------
    mat : numpy ndarray
        GLRL Matrices for 0, 45, 90 and 135 degrees.
    '''
    runLength = max(f.shape)
    mat0 = glrlm_0(f, mask, grayLevel=Ng, runLength=runLength)
    mat45 = glrlm_45(f, mask, grayLevel=Ng, runLength=runLength)
    mat90 = glrlm_90(f, mask, grayLevel=Ng, runLength=runLength)
    mat135 = glrlm_135(f, mask, grayLevel=Ng, runLength=runLength)
    mat = np.dstack((mat0, mat45, mat90, mat135))
    return mat

def glrlm_features(f, mask, Ng=256):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else. Give None
        if you want to consider ROI the whole image.
    Ng : int, optional
        Image number of gray values. The default is 256.
    Returns
    -------
    features : numpy ndarray
        1)Short Run Emphasis, 2)Long Run Emphasis, 3)Gray Level
        Non-Uniformity/Gray Level Distribution, 4)Run Length
        Non-Uniformity/Run Length Distribution, 5)Run Percentage,
        6)Low Gray Level Run Emphasis, 7)High Gray Level Run Emphasis,
        8)Short Low Gray Level Emphasis, 9)Short Run High Gray Level
        Emphasis, 10)Long Run Low Gray Level Emphasis, 11)Long Run
        High Gray Level Emphasis.
    labels : list
        Labels of features.
    '''

    if mask is None:
        mask = np.ones(f.shape)

    labels = ["GLRLM_ShortRunEmphasis",
              "GLRLM_LongRunEmphasis",
              "GLRLM_GrayLevelNo-Uniformity",
              "GLRLM_RunLengthNonUniformity",
              "GLRLM_RunPercentage",
              "GLRLM_LowGrayLevelRunEmphasis",
              "GLRLM_HighGrayLevelRunEmphasis",
              "GLRLM_Short owGrayLevelEmphasis",
              "GLRLM_ShortRunHighGrayLevelEmphasis",
              "GLRLM_LongRunLowGrayLevelEmphasis",
              "GLRLM_LongRunHighGrayLevelEmphasis"]

    rlmatrix = glrlm(f, mask, Ng)

    I, J = _calculate_ij(rlmatrix)
    S = _calculate_s(rlmatrix)
    G = np.apply_over_axes(np.sum, rlmatrix, axes=1)
    R = np.apply_over_axes(np.sum, rlmatrix, axes=0)

    features = np.zeros(11,np.double)
    features[0] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.divide, rlmatrix, (J*J)), axes=(0, 1))[0, 0])/S).mean()
    features[1] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.multiply, rlmatrix, (J*J)), axes=(0, 1))[0, 0])/S).mean()
    features[2] = ((np.apply_over_axes(np.sum, (G*G), axes=(0, 1))[0, 0])/S).mean()
    features[3] = ((np.apply_over_axes(np.sum, (R*R), axes=(0, 1))[0, 0])/S).mean()

    gray_level, run_length,_ = rlmatrix.shape
    num_voxels = gray_level * run_length
    features[4] = (S/num_voxels).mean()

    features[5]= ((np.apply_over_axes(np.sum, _apply_over_degree(np.divide, rlmatrix, (I*I)), axes=(0, 1))[0, 0])/S).mean()
    features[6] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.multiply, rlmatrix, (I*I)), axes=(0, 1))[0, 0])/S).mean()
    features[7] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.divide, rlmatrix, (I*I*J*J)), axes=(0, 1))[0, 0])/S).mean()

    temp = _apply_over_degree(np.multiply, rlmatrix, (I*I))
    features[8] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.divide, temp, (J*J)), axes=(0, 1))[0, 0])/S).mean()

    temp = _apply_over_degree(np.multiply, rlmatrix, (J*J))
    features[9] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.divide, temp, (J*J)), axes=(0, 1))[0, 0])/S).mean()
    features[10] = ((np.apply_over_axes(np.sum, _apply_over_degree(np.multiply, rlmatrix, (I*I*J*J)), axes=(0, 1))[0, 0])/S).mean()

    return features, labels

features =[]


for directory_path in glob.glob("/content/covid_dataset_spilting/test/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        img = cv2.resize(img, (512,512))
        feature,labels=glrlm_features(img,None)
        features.append(feature)

df = pd.DataFrame()

df[labels]  = features

df.to_csv('glrm_features.csv')


In [ ]:
import numpy as np
import cv2
import pandas as pd
import glob
import os

def fos(f, mask):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else. Give None
        if you want to consider ROI the whole image.
    Returns
    -------
    features : numpy ndarray
        1)Mean, 2)Variance, 3)Median (50-Percentile), 4)Mode,
        5)Skewness, 6)Kurtosis, 7)Energy, 8)Entropy,
        9)Minimal Gray Level, 10)Maximal Gray Level,
        11)Coefficient of Variation, 12,13,14,15)10,25,75,90-
        Percentile, 16)Histogram width
    labels : list
        Labels of features.
    '''
    if mask is None:
        mask = np.ones(f.shape)

    # 1) Labels
    labels = ["FOS_Mean","FOS_Variance","FOS_Median","FOS_Mode","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth"]

    # 2) Parameters
    f  = f.astype(np.uint8)
    mask = mask.astype(np.uint8)
    level_min = 0
    level_max = 255
    Ng = (level_max - level_min) + 1
    bins = Ng

    # 3) Calculate Histogram H inside ROI
    f_ravel = f.ravel()
    mask_ravel = mask.ravel()
    roi = f_ravel[mask_ravel.astype(bool)]
    H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]

    # 4) Calculate Features
    features = np.zeros(16,np.double)
    i = np.arange(0,bins)
    features[0] = np.dot(i,H)
    features[1] = sum(np.multiply(((i-features[0])**2),H))
    features[2] = np.percentile(roi,50)
    features[3] = np.argmax(H)
    features[4] = sum(np.multiply(((i-features[0])**3),H))/(np.sqrt(features[1])**3)
    features[5] = sum(np.multiply(((i-features[0])**4),H))/(np.sqrt(features[1])**4)
    features[6] = sum(np.multiply(H,H))
    features[7] = -sum(np.multiply(H,np.log(H+1e-16)))
    features[8] = min(roi)
    features[9] = max(roi)
    features[10] = np.sqrt(features[2]) / features[0]
    features[11] = np.percentile(roi,10)
    features[12] = np.percentile(roi,25)
    features[13] = np.percentile(roi,75)
    features[14] = np.percentile(roi,90)
    features[15] = features[14] - features[11]

    return features, labels

features =[]


for directory_path in glob.glob("D:/major project/ct-scans_preprossed/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        img = cv2.resize(img, (512,512))
        feature,labels=fos(img,None)
        features.append(feature)

df = pd.DataFrame()

df[labels]  = features

df.to_csv('Fos_features.csv')

In [ ]:
#DWT digital wavelet
import pywt
import numpy as np
import math
def _next_power_of_two(n):
    math.ceil(math.log(n,2))
    return 2 ** math.ceil(math.log(n,2))

def _pad_image_power_2(f):
    N1, N2 = f.shape
    N1_deficit = _next_power_of_two(N1) - N1
    N2_deficit = _next_power_of_two(N2) - N2
    f2 = np.pad(f, ((math.floor(N1_deficit/2),N1_deficit-math.floor(N1_deficit/2)),
                    (math.floor(N2_deficit/2),N2_deficit-math.floor(N2_deficit/2))),
                mode='constant')
    return f2

def dwt_features(f, mask, wavelet='bior3.3', levels=3):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else. Give None
        if you want to consider ROI the whole image.
    wavelet : str, optional
         Filter to be used. Check pywt for filter families. The default is 'bior3.3'
    levels : int, optional
        Levels of decomposition. Default is 3.
    Returns
    -------
    features : numpy ndarray
        Mean and std of each detail image. Appromimation images are ignored.
    labels : list
        Labels of features.
    '''

    if mask is None:
        mask = np.ones(f.shape)

    # Step 1: Pad Image
    f = _pad_image_power_2(f)       # pad to the next power of 2 in each dimension
    mask = _pad_image_power_2(mask) # pad to the next power of 2 in each dimension

    # Step 2: Get DWT Decomposition for 3 levels
    coeffs = pywt.wavedec2(f, wavelet=wavelet, level=levels)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    # Step 3: Get DWT Decomposition for 3 levels for mask
    coeffs_mask = pywt.wavedec2(mask, wavelet=wavelet, level=levels)
    coeff_arr_mask, coeff_slices_mask = pywt.coeffs_to_array(coeffs_mask)
    coeff_arr_mask[coeff_arr_mask!=0] = 1

    # Step 4: For each coeff array (10-1=9 sub-images), get mean and std
    labels = []
    features = np.zeros((3*levels,2),np.double)
    i = 0
    for level in range(1,levels+1):
        for name in ['da','dd','ad']:
            D_f =  coeff_arr[coeff_slices[level][name]]
            D_mask = coeff_arr_mask[coeff_slices[level][name]]
            D = D_f.flatten()[D_mask.flatten().astype(bool)] # work on elements inside mask
            features[i][0], features[i][1] = abs(D).mean(), abs(D).std()
            i += 1
            labels.append('DWT_' + str(wavelet) + '_level_' + str(level) +
                          '_' + str(name) + '_mean')
            labels.append('DWT_' + str(wavelet) + '_level_' + str(level) +
                          '_' + str(name) + '_std')

    # Step 5: Return
    return features.flatten(), labels

img = cv2.imread('/content/ctscan_preprocessed/covid_preprocessed/covid_preposed_1.png')

feature ,ld = dwt_features(img,None)
print(feature,ld)




ValueError: ignored

In [ ]:
for directory_path in glob.glob("/content/ctscan_preprocessed/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        img = cv2.resize(img, (512,512))
        feature,labels=dwt_features(img,None)
        features.append(feature)

df = pd.DataFrame()

df[labels]  = features

df.to_csv('dwt_features.csv')

In [ ]:
#hog feature
from skimage import feature

def hog_features(f):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    ppc : int, optional
        Pixels per cell. The default is 8.
    cpb : int, optional
        Cells per block. The default is 3.
    Returns
    -------
    fd : numpy ndarray
        Histogram of Oriented Gradients flattened.
    labels : list
        Labels of features.
    '''

    fd, _ = feature.hog(f, orientations=9,  block_norm='L2', visualize=True)
    labels = [('HOS_' + str(i)) for i in range(fd.shape[0])]
    return fd, labels
features = []
cout=0
for directory_path in glob.glob("/content/ctscan_preprocessed/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        img = cv2.resize(img, (512,512))
        fea,labels=hog_features(img)
        features.append(fea)
        cout+=1
        print("feature extraction done",cout)

df = pd.DataFrame()

df[labels]  = features

df.to_csv('hog_features.csv')

feature extraction done 1
feature extraction done 2
feature extraction done 3
feature extraction done 4
feature extraction done 5
feature extraction done 6
feature extraction done 7
feature extraction done 8
feature extraction done 9
feature extraction done 10
feature extraction done 11
feature extraction done 12
feature extraction done 13
feature extraction done 14
feature extraction done 15
feature extraction done 16
feature extraction done 17
feature extraction done 18
feature extraction done 19
feature extraction done 20
feature extraction done 21
feature extraction done 22
feature extraction done 23
feature extraction done 24
feature extraction done 25
feature extraction done 26
feature extraction done 27
feature extraction done 28
feature extraction done 29
feature extraction done 30
feature extraction done 31
feature extraction done 32
feature extraction done 33
feature extraction done 34
feature extraction done 35
feature extraction done 36
feature extraction done 37
feature ex

In [ ]:
# Gabour feature
import cv2
import glob
import os
import pandas as pd


import numpy as np
from skimage.filters import gabor_kernel
from scipy import signal

def _image_xor(f):
    # Turn "0" to "1" and vice versa: XOR with image consisting of "1"s
    f = f.astype(np.uint8)
    mask = np.ones(f.shape, np.uint8)
    out = np.zeros(f.shape, np.uint8)
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            out[i,j] = f[i,j] ^ mask[i,j]
    return out

def gt_features(f, mask, deg=4, freq=[0.05, 0.4]):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else. Give None
        if you want to consider ROI the whole image.
    deg: int, optinal
        Quantized degrees. The default is 4 (0, 45, 90, 135 degrees)
    freq: list, optional
        frequency of the gabor kernel. The default is [0.05, 0.4]
    Returns
    -------
    features : numpy ndarray
        Mean and std for the resulted image: (f o gabor_filter)(x,y)

    labels : list
        Labels of features.
    '''

    if mask is None:
        mask = np.ones(f.shape)

    # Step 1: Initialize kernels
    kernels = []
    labels = []
    for theta in range(deg): # e.g. th = 0, 45, 90, 135 degrees
        theta = theta / deg * np.pi
        for frequency in freq: # e.g. f = 0.05 and 0.4
            kernel = np.real(gabor_kernel(frequency, theta=theta))
            kernels.append(kernel)
            labels.append('GT_th_' + str(theta*4/np.pi) + '_freq_' +
                          str(frequency) + '_mean')
            labels.append('GT_th_' + str(theta*4/np.pi) + '_freq_' +
                          str(frequency) + '_std')

    # Step 2: Get mask where convolution should be performed
    mask_c = _image_xor(mask)
    mask_convs = []
    for k in range(len(kernels)):
        oneskernel = np.ones(kernels[k].shape)
        temp = signal.convolve2d(mask_c, oneskernel,'same')
        temp = np.abs(np.sign(temp)-1)
        mask_convs.append(temp)

    # Step 3: Convolve image with each kernel and get mean and std
    features = np.zeros((len(kernels), 2), np.double)
    for k in range(len(kernels)):
            D = signal.convolve2d(f, kernels[k], 'same')
            D = np.multiply(D, mask_convs[k])
            D_ravel = D.ravel()
            mask_conv_ravel = mask_convs[k].ravel()
            roi = D_ravel[mask_conv_ravel==1]
            if roi.size == 0:
                features[k, 0], features[k, 1] = 0, 0
            else:
                features[k, 0], features[k, 1] = roi.mean(), roi.std()

    # step 4: Return
    return features.flatten(), labels


features = []
cout=0
for directory_path in glob.glob("/content/ctscan_preprocessed/*"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path,0)
        img = cv2.resize(img, (512,512))
        fea,labels=gt_features(img,None)
        features.append(fea)
        cout+=1
        print("feature extraction done",cout)

df = pd.DataFrame()

df[labels]  = features

df.to_csv('gt_features.csv')

feature extraction done 1
feature extraction done 2
feature extraction done 3
feature extraction done 4
feature extraction done 5
feature extraction done 6
feature extraction done 7
feature extraction done 8
feature extraction done 9
feature extraction done 10
feature extraction done 11
feature extraction done 12
feature extraction done 13
feature extraction done 14
feature extraction done 15
feature extraction done 16
feature extraction done 17
feature extraction done 18
feature extraction done 19
feature extraction done 20
feature extraction done 21
feature extraction done 22
feature extraction done 23
feature extraction done 24
feature extraction done 25
feature extraction done 26
feature extraction done 27
feature extraction done 28
feature extraction done 29
feature extraction done 30
feature extraction done 31
feature extraction done 32
feature extraction done 33
feature extraction done 34
feature extraction done 35
feature extraction done 36
feature extraction done 37
feature ex